In [1]:
import os

if "src" not in os.listdir():
    os.chdir("../")

In [3]:
import numpy as np

import pandas as pd
from loguru import logger
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from qdrant_client import QdrantClient, models
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10

from src.conf import url_qdrant


output_notebook()

ModuleNotFoundError: No module named 'qdrant_client'

In [ ]:
path_save_points_for_visual = "./data/interim/points_for_visual.csv"

client = QdrantClient(url=url_qdrant)
collection_name = "911_hybrid_rating_points"
num_examples_for_visual = 10_000

In [ ]:
if (
    os.path.exists(path_save_points_for_visual)
    and len(pd.read_csv(path_save_points_for_visual)) == num_examples_for_visual
):
    logger.info(f"Будут использованы points с {path_save_points_for_visual}")
    ids_points = pd.read_csv(path_save_points_for_visual)["id"].to_list()
    points = client.retrieve(
        collection_name=collection_name,
        ids=ids_points,
        with_vectors=True,
        timeout=10_000,
    )
else:
    logger.info(f"Будет сформирована случайная выборка points")
    points = client.query_points(
        collection_name=collection_name,
        query=models.SampleQuery(sample=models.Sample.RANDOM),
        limit=num_examples_for_visual,
        with_vectors=True,
        timeout=10_000,
    ).points

    ids_points = [i.id for i in points]

    pd.DataFrame({"id": ids_points}).to_csv(path_save_points_for_visual)

In [ ]:
embeddings = []
texts = []
ids = []

for i in points:
    embeddings.append(i.vector)
    texts.append(i.payload["question"])
    ids.append(i.id)

In [ ]:
def reduce_dimensions(embeddings, n_components=2, perplexity=30, learning_rate=200):
    # Используем t-SNE для уменьшения размерности
    reducer = TSNE(
        n_components=n_components,
        perplexity=perplexity,
        learning_rate=learning_rate,
        random_state=42,  # Фиксируем random_state для детерминированности
    )
    return reducer.fit_transform(embeddings)


def cluster_embeddings(reduced_embeddings, n_clusters=5):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    return kmeans.fit_predict(reduced_embeddings)


def visualize_embeddings(reduced_embeddings, texts, ids, cluster_labels):
    df = pd.DataFrame(
        {
            "x": reduced_embeddings[:, 0],
            "y": reduced_embeddings[:, 1],
            "text": texts,
            "id": ids,
            "cluster": cluster_labels,
        }
    )

    # Создаем палитру цветов
    unique_clusters = df["cluster"].unique()
    print(unique_clusters)
    print(len(unique_clusters))
    palette = Category10[len(unique_clusters)]
    color_map = {cluster: color for cluster, color in zip(unique_clusters, palette)}

    # Добавляем столбец с цветами в DataFrame
    df["fill_color"] = df["cluster"].apply(lambda x: color_map[x])

    # Создаем источник данных
    source = ColumnDataSource(df)

    # Настройка инструментов
    TOOLS = "hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,examine,help"

    # Создание графика
    p = figure(
        title="Визуализация эмбеддингов с кластерами",
        x_axis_label="x",
        y_axis_label="y",
        tools=TOOLS,
        output_backend="webgl",
        width=1200,
        height=1200,
    )  # Использование WebGL для лучшей производительности

    # Добавление точек на график
    p.scatter(
        "x",
        "y",
        size=5,  # Размер точки
        fill_color="fill_color",
        fill_alpha=0.6,
        line_color=None,
        source=source,
    )

    # Настройка отображения текста при наведении
    hover = HoverTool(
        tooltips=[
            ("Text", "@text"),
            ("Cluster", "@cluster"),
        ],
        mode="mouse",
    )

    p.add_tools(hover)

    # Отображение графика
    show(p)


# Сжатие эмбеддингов до 2D
reduced_embeddings = reduce_dimensions(np.array([i["dense"] for i in embeddings]), n_components=2)

# Кластеризация
cluster_labels = cluster_embeddings(reduced_embeddings, n_clusters=10)

# Визуализация
visualize_embeddings(reduced_embeddings, texts, ids, cluster_labels)

In [ ]:
unique_clusters = [8, 5, 6, 7, 2, 0, 3, 9, 1, 4]
Category10[len(unique_clusters)]

In [ ]:
len(unique_clusters)

In [ ]:
Category10[1]

Эмбеддинги не идеальные. Видно, что они переодически группируются не по юридическим основаниям, а по упоминанию материальных объектов (квартира, машина). Хотя чаще эмбеддинги адекватные.